In [1]:
import numpy as np
from bokeh.plotting import *
from bokeh.models import ColumnDataSource
import pandas as pd

In [2]:
output_notebook()
output_file("linked_brushing.html", title='Bokeh demo 1')

Loading BokehJS ...

# Making plot for only 1 station

In [37]:
# all epa stations
file = '~/CMAQcheck/NO2_d02_2018_8_EPA_CMAQ_Combine.csv'
file2 = '~/CMAQcheck/NO2_d03_2018_8_EPA_CMAQ_Combine.csv'
df=pd.read_csv(file)
df2=pd.read_csv(file2)

# Cut down to one station
sitenum=56
df=df[df['Site Num']==sitenum]
df2=df2[df2['Site Num']==sitenum]

In [38]:
y0,ym,t=df['Sample Measurement'],df['CMAQ'],pd.to_datetime(df['level_0'])
ym2,t2=df2['CMAQ'],pd.to_datetime(df2['level_0'])

In [39]:
from bokeh.layouts import gridplot
from bokeh.plotting import figure, output_file, show


def datetime(x):
    return np.array(x, dtype=np.datetime64)

p1 = figure(x_axis_type="datetime", title="August NO2 at Station %i"%(sitenum))
p1.grid.grid_line_alpha=0.3
p1.xaxis.axis_label = 'Date'
p1.yaxis.axis_label = 'Concentration (ppb)'

p1.line(datetime(t2), ym2, color='#33A02C', legend_label='CMAQ_d03')
p1.line(datetime(t), ym, color='#A6CEE3', legend_label='CMAQ_d02')
p1.circle(datetime(t), y0, color='#B2DF8A', legend_label='EPA')

output_file("onestation.html", title="One EPA Time Series")
show(p1)

# Making plot for all stations, choose station by making dropdown option

In [99]:
from bokeh.io import show
from bokeh.models import CustomJS, Dropdown, MultiChoice, Select
from bokeh.layouts import column

In [125]:
# Pick the chemical and date -- O3, CO, SO2, NO2

#NO2_d02_2018_8_EPA_CMAQ_Combine.csv

var='SO2'
month='8'
year='2018'


In [126]:
# all epa stations
file = '~/CMAQcheck/%s_d02_%s_%s_EPA_CMAQ_Combine.csv'%(var,year,month)
file2 = '~/CMAQcheck/%s_d03_%s_%s_EPA_CMAQ_Combine.csv'%(var,year,month)
df=pd.read_csv(file)
df2=pd.read_csv(file2)

df['Date']=pd.to_datetime(df['level_0'])
df2['Date']=pd.to_datetime(df2['level_0'])

if var=='CO':
    df['Sample Measurement']=df['Sample Measurement']*1000
    df2['Sample Measurement']=df2['Sample Measurement']*1000
elif var=='O3':
    df['Sample Measurement']=df['Sample Measurement']*1000
    df2['Sample Measurement']=df2['Sample Measurement']*1000


latlon=[str(df.Latitude[i]) + " " + str(df.Longitude[i]) for i in range(len(df))]
latlon2=[str(df2.Latitude[i]) + " " + str(df2.Longitude[i]) for i in range(len(df2))]

df['latlon']=latlon;
df2['latlon']=latlon2;


m=pd.merge(df2,df,on=['latlon','Date','Site Num'],suffixes=('_d02', '_d03'))

/Users/anastasiamontgomery/miniconda3/envs/dataviz/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3146: DtypeWarning: Columns (20) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [127]:
sites[0]

'1'

In [128]:
sites=m['Site Num'].unique()
sites=sites[~np.isnan(sites)].astype(int)
sites.sort()
sites=sites.astype(str).tolist()

In [129]:
# Filter out original data frame for useful info ONLY
gap1=m.loc[:, ['Site Num','Sample Measurement_d02','CMAQ_d02','CMAQ_d03', 'Date']]
gap2 = gap1[gap1['Site Num'] == int(sites[0])]

# Make total df, and current df
Overall = ColumnDataSource(data=gap1)
Curr=ColumnDataSource(data=gap2)

#plot and the menu is linked with each other by this callback function
callback = CustomJS(args=dict(source=Overall, sc=Curr), code="""
var f = cb_obj.value
sc.data['Date']=[]
sc.data['Sample Measurement_d02']=[]
sc.data['CMAQ_d03']=[]
sc.data['CMAQ_d02']=[]
for(var i = 0; i <= source.get_length(); i++){
    if (source.data['Site Num'][i] == f){
        sc.data['Date'].push(source.data['Date'][i])
        sc.data['Sample Measurement_d02'].push(source.data['Sample Measurement_d02'][i])
        sc.data['CMAQ_d02'].push(source.data['CMAQ_d02'][i])
        sc.data['CMAQ_d03'].push(source.data['CMAQ_d03'][i])
    }
}   
   
sc.change.emit();
""")

# drop down menu
menu = Select(options=sites,value='56', title = 'Station Number') 

t=month+'/'+year+" "+var+" at Station"

#creating figure object
p=figure(x_axis_type="datetime", title=t) 

p.grid.grid_line_alpha=0.3
p.xaxis.axis_label = 'Date'
p.yaxis.axis_label = 'Concentration (ppb)'

p.line(x='Date', y='CMAQ_d02', color='#33A02C', legend_label='CMAQ_d02',source=Curr)
p.line(x='Date', y='CMAQ_d03',color='#A6CEE3', legend_label='CMAQ_d03',source=Curr)
p.circle(x='Date', y='Sample Measurement_d02', color='#B2DF8A', legend_label='EPA',source=Curr)

# calling the function on change of selection
menu.js_on_change('value', callback) 

# creating the layout
layout=column(menu, p) 

show(layout)

# ---> WISHES : Had a map of the model domain with stations highlighted as we click through stations
# ---> Choose different chemical constituents
# ---> Methodology of each station listed ....

# Takeaways 
- 17 is weird 

- 10 has 2 stations in it


# Make a map with stations, hover over the station and give the values